In [ ]:
#!jupyter nbconvert --to=python DIN_WOA_Comparison_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class DINcomp:
    '''
    class DINcomp
    
    compare FESOM-REcoM DIN data to World Ocean Atlas 
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 WOAvar='n_an',
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,
                 depth_array=(0,50,200,1000,2000,4000),
                 depth_limit=100,
                 cmap_extension='max',
                 verbose=True,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 get_overview = False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit
        self.cmap_extension = cmap_extension
        self.get_overview = get_overview
        self.verbose = verbose
        self.plotting = plotting
        self.Taylor = Taylor
        
        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        #from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from load_interp_WOA_python3 import WOAdata
        from plot_Taylor_normalized import plt_Taylor_norm
        from plot_Taylor_normalized import plt_Taylor_comp
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)
        
        # load FESOM data -------------------------------------------------------------------------------------
        #print('Processing ' + self.resultpath)
        
        #mesh       = pf.load_mesh(self.meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DIN.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'DIN [mmol m$^{-3}$]' # equals to mumol/L

        # load data -------------------------------------------------------------------------------------
        DINfesom = pf.get_data(self.resultpath, "DIN", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)

        self.DINfesom = DINfesom

        # load WOA data  -------------------------------------------------------------------------------------
        WOA_input = WOAdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.WOAvar, get_overview=False)
        din_int = WOA_input.woa_int    
        #print(np.shape(din_int))
        
        labelwoa = 'WOA'
        unitwoa = 'DIN [mmol m$^{-3}$]'

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        din_int_ma = np.copy(din_int)
        din_int_ma[DINfesom == 0] = 0

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(self.verbose):
                    print('\nInput depth = {0}, plotting at depth = {1} m\nFESOM min = {2}, max = {3}\nWOA min = {4}, max = {5}'.format(
                        d,plot_depth,
                        np.nanmin(DINfesom[:,i]),np.nanmax(DINfesom[:,i]),
                        np.nanmin(din_int_ma[:,i]),np.nanmax(din_int_ma[:,i])))
                    print('Min of difference WOA-FESOM: {0}'.format(np.nanmin(din_int_ma[:,i]-DINfesom[:,i])))
                
                if plotting:
                    fig = plt.figure(figsize=(15,15), constrained_layout=True)
                    axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                                subplot_kw=dict(projection=self.mapproj))

                    m1 = axes['A']
                    levels = np.arange(0,25,1)
                    f1 = pf.subplot(mesh, m1, [DINfesom[:,i]],
                                levels = levels,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelfesom+'\n ({0} m)'.format(depth_limit),
                               )

                    m2 = axes['B']
                    f2 = pf.subplot(mesh, m2, [din_int_ma[:,i]], 
                                levels = levels,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelwoa+'\n ({0} m)'.format(depth_limit),
                               )

                    cbar1_ax = fig.add_axes([0.13, 0.55, 0.76, 0.02])
                    cbar1 = fig.colorbar(f1,
                                    cax = cbar1_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar1.set_label(unitfesom, fontsize=18)
                    cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-15,16,2)
                    f3 = pf.subplot(mesh, m3, [DINfesom[:,i]-din_int_ma[:,i]], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                titles='FESOM - WOA\n ({0} m)'.format(depth_limit),
                               )

                    fig.subplots_adjust(bottom=0.02)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'DIN_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}m.png'.format(depth_limit), 
                                dpi = 300, bbox_inches='tight')
                    plt.show(block=False)
                    
                if Taylor:
                    # statistics  -------------------------------------------------------------------------------------
                    # preparation of datasets
                    if np.isnan(np.min(din_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
                    if np.isnan(np.min(DINfesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                    title = 'Normalized Taylor Diagram for DIN'
                    plt_Taylor_comp(din_int_ma,DINfesom,mask=True,title=title, depth_array=depth_array, mesh=mesh)
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'DIN_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_layerwise.png', 
                                        dpi = 300, bbox_inches='tight')
                    plt.show(block=False) 
                    
                if output:
                    print('Only return non-layerwise output')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            # get level depth index closest to depth_limit
            i_depth_limit = pf.ind_for_depth(self.depth_limit, mesh)
            
            # corresponding level depth
            depth_limit_level = - mesh.zlev[i_depth_limit]
            
            # corresponding layer depth (where concentrations are defined):
            depth_limit_layer = - WOA_input.layer_depths[i_depth_limit]
            
            if(self.verbose):
                # print overview of chosen depth
                print('DIN as mean over depth \nwith max layer depth = {0} \n(level depth = {1}m, mesh index {2})'.format(
                    depth_limit_layer,depth_limit_level,i_depth_limit))
            
            # mean over depth
            DINfesom_mean = np.mean(DINfesom[:,:i_depth_limit], axis = 1)
            din_int_ma_mean = np.mean(din_int_ma[:,:i_depth_limit], axis = 1)
            #print(np.shape(DINfesom_mean), np.shape(din_int_ma_mean))
            
            self.DINfesom_mean = DINfesom_mean
            self.din_int_ma_mean = din_int_ma_mean
            
            if(self.verbose):
                print('\nFESOM min = {0}, max = {1}\nWOA min = {2}, max = {3}'.format(
                        np.nanmin(DINfesom_mean),np.nanmax(DINfesom_mean),
                        np.nanmin(din_int_ma_mean),np.nanmax(din_int_ma_mean),
                        depth_limit_layer))
                print('Min of difference WOA-FESOM: {0}'.format(np.nanmin(din_int_ma_mean-DINfesom_mean)))
            
            if plotting:
                fig = plt.figure(figsize=(15,15), constrained_layout=True)
                axes = fig.subplot_mosaic(
                            """
                            AB
                            CC
                            """,
                            gridspec_kw={'hspace': 0.1, 'wspace': 0.1}, 
                            subplot_kw=dict(projection=self.mapproj))
                    
                m1 = axes['A']
                levels = np.arange(0,25,1)
                f1 = pf.subplot(mesh, m1, [DINfesom_mean],
                            levels = levels,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelfesom+'\n (0-{0} m)'.format(depth_limit),
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, m2, [din_int_ma_mean], 
                            levels = levels,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelwoa+'\n (0-{0} m)'.format(depth_limit),
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.55, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-15,16,1)
                f3 = pf.subplot(mesh, m3, [DINfesom_mean-din_int_ma_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles='FESOM - WOA'+'\n (0-{0} m)'.format(depth_limit),
                           )
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'DIN_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False)
                
            if Taylor:
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(din_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
                if np.isnan(np.min(DINfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                title = 'Taylor Diagram for DIN \n(mean over depth, max = {0}m)'.format(depth_limit_layer)
                plt_Taylor_norm(din_int_ma_mean,DINfesom_mean,mask=True,title=title,verbose=True)

                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'DIN_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False)
                
            if output:
                    self.din_fesom = DINfesom_mean
                    self.din_phc   = din_int_ma_mean